In [2]:
# -*- coding: utf-8 -*-
import sys
from pymarc import Field, Reader, Record, MARCWriter
!pip install elasticsearch==7.6.0
!pip install pymongo
!pip install pymarc

You should consider upgrading via the '/Users/vyacheslavtykhonov/.pyenv/versions/3.7.0/envs/jupyter3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/vyacheslavtykhonov/.pyenv/versions/3.7.0/envs/jupyter3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/vyacheslavtykhonov/.pyenv/versions/3.7.0/envs/jupyter3/bin/python -m pip install --upgrade pip' command.


In [3]:
!curl 'https://storage.googleapis.com/kaggle-data-sets/551982%2F1359228%2Fcompressed%2Fmetadata.csv.zip?GoogleAccessId=gcp-kaggle-com@kaggle-161607.iam.gserviceaccount.com&Expires=1595663670&Signature=nviUFqLPLOXX32rNnjQoali3UKgteYSsvdqzJPFfJ8WSILtHMSqU7C5seb%2FeAc96fJkmdBOf5OoVWwpHwt4r%2FnOUt3NZkxf63aR6PLh8vieD4CloYtqGOEIb8Pg6wNG3LwUD8fIE0jR42WjYa%2FVkYshsBASR1Mz8D6z2TW1DPIDp%2Bdjw%2BEXKjRqjQ%2FXdN1hDjZ2uoQ%2FmCWdZAf83G%2B716TujB5G4LrlA23ReGVxlFZLLeycnRTeYrkY3TrDZVSTwpAWKiC5pRe7aeQ4i87I3%2BpQj0RO%2FxRWI2ZzSlacbfDr0lq8EpLQ52jSmkEcd%2F2zN4qWKRClNbhmScJp8Mkvf%2BA%3D%3D' -H 'User-Agent: Mozilla/5.0 (X11; Linux x86_64; rv:79.0) Gecko/20100101 Firefox/79.0' -H 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8' -H 'Accept-Language: en-US,en;q=0.5' --compressed -H 'Referer: https://www.kaggle.com/' -H 'Connection: keep-alive' -H 'Upgrade-Insecure-Requests: 1' -H 'TE: Trailers' -o compressed_metedata.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 94.7M  100 94.7M    0     0  15.9M      0  0:00:05  0:00:05 --:--:-- 18.7M


In [4]:
!rm metadata.csv
!unzip compressed_metedata.zip

Archive:  compressed_metedata.zip
  inflating: metadata.csv            


In [5]:
import pandas as pd
metadata = pd.read_csv('metadata.csv')
metadata.info()

/Users/vyacheslavtykhonov/.pyenv/versions/jupyter3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (1,4,5,6,13,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195711 entries, 0 to 195710
Data columns (total 19 columns):
cord_uid            195711 non-null object
sha                 82517 non-null object
source_x            195711 non-null object
title               195655 non-null object
doi                 139395 non-null object
pmcid               87299 non-null object
pubmed_id           119076 non-null object
license             195711 non-null object
abstract            139874 non-null object
publish_time        195693 non-null object
authors             189141 non-null object
journal             184692 non-null object
mag_id              0 non-null float64
who_covidence_id    51738 non-null object
arxiv_id            2500 non-null object
pdf_json_files      82517 non-null object
pmc_json_files      63394 non-null object
url                 153320 non-null object
s2_id               167757 non-null float64
dtypes: float64(2), object(17)
memory usage: 28.4+ MB


In [6]:
metadata.head()

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
0,ug7v899j,d1aafb70c066a2068b02786f8929fd9c900897fb,PMC,Clinical features of culture-proven Mycoplasma...,10.1186/1471-2334-1-6,PMC35282,1.14726e+07,no-cc,OBJECTIVE: This retrospective chart review des...,2001-07-04,"Madani, Tariq A; Al-Ghamdi, Aisha A",BMC Infect Dis,NaN,NaN,NaN,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN
1,02tnwd4m,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d,PMC,Nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,PMC59543,1.1668e+07,no-cc,Inflammatory diseases of the respiratory tract...,2000-08-15,"Vliet, Albert van der; Eiserich, Jason P; Cros...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
2,ejv2xln0,06ced00a5fc04215949aa72528f2eeaae1d58927,PMC,Surfactant protein-D and pulmonary host defense,10.1186/rr19,PMC59549,1.1668e+07,no-cc,Surfactant protein-D (SP-D) participates in th...,2000-08-25,"Crouch, Erika C",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/06ced00a5fc04215949aa...,document_parses/pmc_json/PMC59549.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
3,2b73a28n,348055649b6b8cf2b9a376498df9bf41f7123605,PMC,Role of endothelin-1 in lung disease,10.1186/rr44,PMC59574,1.16869e+07,no-cc,Endothelin-1 (ET-1) is a 21 amino acid peptide...,2001-02-22,"Fagan, Karen A; McMurtry, Ivan F; Rodman, David M",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/348055649b6b8cf2b9a37...,document_parses/pmc_json/PMC59574.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
4,9785vg6d,5f48792a5fa08bed9f56016f4981ae2ca6031b32,PMC,Gene expression in epithelial cells in respons...,10.1186/rr61,PMC59580,1.16869e+07,no-cc,Respiratory syncytial virus (RSV) and pneumoni...,2001-05-11,"Domachowske, Joseph B; Bonville, Cynthia A; Ro...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/5f48792a5fa08bed9f560...,document_parses/pmc_json/PMC59580.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN


In [7]:
item = metadata.ix[0]
len(metadata.index)

/Users/vyacheslavtykhonov/.pyenv/versions/jupyter3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


195711

In [23]:
OUTFILE = '/tmp/cord.mrc'
#writer = MARCWriter(file(OUTFILE, 'wb'))
writer = MARCWriter(open(OUTFILE,'wb'))
#create a marc record instance
#for i in metadata.index[0:10000000]:
for i in reversed(range(len(metadata.index))):
#    print(i)
	title = metadata.ix[i]['title']
	authors = ''
	try:
		authors = metadata.ix[i]['authors'].split('; ')
	except:
		continue
	abstract = metadata.ix[i]['abstract']    
	isbn = metadata.ix[i]['cord_uid']
	imprint = metadata.ix[i]['pmcid']
	imprint=''
	license = metadata.ix[i]['license']
	year = str(metadata.ix[i]['publish_time'])[0:4]
	ctrl = metadata.ix[i]['pmcid'] #findvalue(rownum, 2)
	language = 'eng'    
	doi="https://doi.org/%s" % metadata.ix[i]['doi']
	subject='Medicine'
	tag2 = 'coronavirus'    
	affiliations = ["CoronaWhy"]
    
	record = Record()#to_unicode=True)
	
	#store vendor catalog id in 001
	field = Field(tag='001', data=ctrl)
	record.add_field(field)
	
	#008
	oh8 = list('/' * 39)
	oh8[35:37] = 'chi'
	oh8[15:17] = 'cc '
	field = Field(tag='008', data="".join(oh8))
	record.add_field(field)
	
	#title
	field = Field('245', ['1', '0'], ['a', title])
	record.add_field(field)
	#leader
	l = list(record.leader)
	l[5] = 'n'
	l[6] = 'a'
	l[7] = 'm'
	l[9] = 'a' # flag saying this record is utf8
	record.leader = "".join(l)
	
	if doi: 
		field = Field('856', ['40', ''], ['u', doi])
		record.add_field(field)
	#tags
	if tag2:
		field = Field('651', [' ', ' '], ['x', tag2])
		record.add_field(field)
	
	field = Field(
		tag = '260',
		indicators = ['\\', '\\'],
		subfields = [
			'b', imprint, 
			'c', year
			]
		)
	record.add_field(field)
	if isbn:
		field = Field('020', [' ', ' '], ['a', isbn])
		record.add_field(field)
	
	#language
	field = Field('041', [' ', ' '], ['a', language])    
	record.add_field(field)
	
	#author
	for a in authors:
		field = Field('100', ['0', ' '], ['a', a], ['u', str(affiliations)])
		record.add_field(field)
	
	#subject
	field = Field('650', ['00', ' '], ['a', subject])    
	record.add_field(field)
	
	#title
	if title:
		field = Field('245', ['1', '0'], ['a', title])
		record.add_field(field)
	
	#affiliation 852$a
	for aff in affiliations:
		field = Field('852', ['', ''], ['a', aff])
		record.add_field(field)
	# license
	if license:    
		field = Field('900', [' ', ' '], ['a', license])
		record.add_field(field)
	if abstract:
		field = Field('520', ['1', '0'], ['a', abstract])
		record.add_field(field)
	#write the record to file
	#writer.write(record)
	try:
		xcheck = record.as_marc()
		writer.write(record)
	except:
		#print(record.as_marc())
		#break #continue
		continue
#	print(record)
writer.close()
print('Ready!')

/Users/vyacheslavtykhonov/.pyenv/versions/jupyter3/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  
/Users/vyacheslavtykhonov/.pyenv/versions/jupyter3/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  # This is added back by InteractiveShellApp.init_path()
/Users/vyacheslavtykhonov/.pyenv/versions/jupyter3/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.or

Ready!
